In [1]:
from collections import deque

import numpy as np
import progressbar as pb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym

from environments import *

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
env = gym.make('G1Dist-v0', new_step_api=True)

In [4]:
# reset the environment
env_info = env.reset()

# size of each action
action_size = env.action_space.shape[0]
print('Size of each action:', action_size)

state_size = env.observation_space.shape[0]
print('Observation size: {}'.format(state_size))

Size of each action: 12
Observation size: 35


In [5]:
NUM_AGENTS = 1

def interact(action):
    action = action.reshape(NUM_AGENTS, action_size)
    next_state, reward, done, trunc, info = env.step(action)
    return next_state.reshape(NUM_AGENTS, -1), np.array(reward).reshape(NUM_AGENTS, -1), np.array(done or trunc).reshape(NUM_AGENTS, -1)

def reset():
    state = env.reset().reshape(NUM_AGENTS, -1)
    return state

In [6]:
NET_SIZE = 512

class Policy(nn.Module):
    
    def __init__(self, state_size, action_size=1, n_agents=NUM_AGENTS, fc1_size=NET_SIZE, fc2_size=NET_SIZE):
        super(Policy, self).__init__()
        
        self.bn0 = nn.BatchNorm1d(state_size)
        self.fc1 = nn.Linear(state_size, fc1_size)
        self.bn1 = nn.BatchNorm1d(fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        self.bn2 = nn.BatchNorm1d(fc1_size)
        self.fc3_mu = nn.Linear(fc2_size, action_size)
        self.fc3_std = nn.Linear(fc2_size, action_size)

    def forward(self, state, log_std_min=-20, log_std_max=2):
        x = self.bn0(state)
        x = torch.relu(self.bn1(self.fc1(state)))
        x = torch.relu(self.bn2(self.fc2(x)))

        mean = self.fc3_mu(x)
        std = self.fc3_std(x)
        std = torch.clamp(std, log_std_min, log_std_max).exp()

        return mean, std
    
class Value(nn.Module):
    
    def __init__(self, state_size, action_size=1, n_agents=NUM_AGENTS, fc1_size=NET_SIZE, fc2_size=NET_SIZE):
        
        super(Value, self).__init__()
        
        self.bn0 = nn.BatchNorm1d(state_size)
        self.fc1 = nn.Linear(state_size, fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        self.fc3 = nn.Linear(fc2_size, 1)
        
    def forward(self, x):
        x = self.bn0(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)
    
class Q(nn.Module):
    
    def __init__(self, state_size, action_size, n_agents=NUM_AGENTS, fc1_size=NET_SIZE, fc2_size=NET_SIZE):
        
        super(Q, self).__init__()      

        self.bn0 = nn.BatchNorm1d(state_size+action_size)
        self.fc1 = nn.Linear(state_size + action_size, fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        self.fc3 = nn.Linear(fc2_size, 1)
        
    def forward(self, s, a):
        x = torch.cat([s, a], 1)
        x = self.bn0(x)
        x = torch.relu(self.fc1(x)) 
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [7]:
from algorithms.sac import Agent

agent = Agent(
    state_size=state_size, 
    action_size=action_size,
    policy_network=Policy,
    value_network=Value,
    q_network=Q,
    n_agents=NUM_AGENTS, 
    device=device,
)

c:\Users\Sergei\Anaconda3\envs\pytorch\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [8]:
# agent.policy_network.load_state_dict(torch.load("./trained_models/g1-forward/POLICY.pth"))
# agent.value_network_local.load_state_dict(torch.load("./trained_models/g1-forward/VALUE_TARGET.pth"))
# agent.value_network_target.load_state_dict(torch.load("./trained_models/g1-forward/VALUE_TARGET.pth"))
# agent.q_network_1.load_state_dict(torch.load("./trained_models/g1-forward/Q_1.pth"))
# agent.q_network_2.load_state_dict(torch.load("./trained_models/g1-forward/Q_2.pth"))

In [13]:
env.render()

c:\Users\Sergei\Anaconda3\envs\pytorch\lib\site-packages\gym\core.py:57: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


In [9]:
def run(n_episodes, t_max, print_every):
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=print_every)  # last 100 scores
    widget = ['training loop: ', pb.Percentage(), ' ', pb.Bar(), ' ', pb.ETA()]
    timer = pb.ProgressBar(widgets=widget, maxval=n_episodes).start()
    
    frame_counter = 0
    
    for i_episode in range(1, n_episodes+1):
        states = reset()
        score = 0
        for t in range(t_max):
            frame_counter += 1
            actions = agent.act(states)
            next_states, rewards, dones = interact(actions)
            agent.step(states, actions, rewards, next_states, dones)
            states = next_states
            score += rewards.mean()
            if np.any(dones):
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        agent.writer.add_scalar('score/mean', score, i_episode)
        if i_episode % print_every == 0:
            print('\rEpisode {}\tScore Mean: {:.2f}\tScore STD: {:.2f}'.format(i_episode, np.mean(scores_window), np.std(scores_window)))
            
        timer.update(i_episode)
    return scores

In [12]:
%time scores = run(t_max=int(10000), n_episodes=int(5000), print_every=100)

Episode 30	Score Mean: -51.05	Score STD: 58.51


Episode 60	Score Mean: -43.10	Score STD: 34.32


Episode 90	Score Mean: -8.29	Score STD: 14.43


Episode 120	Score Mean: -19.78	Score STD: 21.07


Episode 150	Score Mean: -30.88	Score STD: 26.64


Episode 180	Score Mean: -14.66	Score STD: 20.86


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
torch.save(agent.policy_network.state_dict(), "./trained_models/g1-forward/POLICY.pth")
torch.save(agent.value_network_target.state_dict(), "./trained_models/g1-forward/VALUE_TARGET.pth")
torch.save(agent.q_network_1.state_dict(), "./trained_models/g1-forward/Q_1.pth")
torch.save(agent.q_network_2.state_dict(), "./trained_models/g1-forward/Q_2.pth")

In [ ]:
env.close()